# A ~5 minute guide to Numba

這邊就是簡單快速地介紹 Numba 是怎麼做的，主要都是參考寫在 README 中的 Reference 網站

In [1]:
import time
import numpy as np
import pandas as pd
from numba import jit

## 1. Will Numba work for my code?

這邊主要是說明，要是你的程式碼應用在數值計算並且當中大量地使用迴圈，那使用 Numba 是你不錯的選擇。
那下面程式碼主要是說明使用 Numba 中的 `@jit` 裝飾器去用在 NumPy 和 Pandas 上。

In [2]:
x = np.arange(100).reshape(10, 10)

@jit(nopython=True)               # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a):                   # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

print(go_fast(x))

[[  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.  25.  26.  27.  28.]
 [ 29.  30.  31.  32.  33.  34.  35.  36.  37.  38.]
 [ 39.  40.  41.  42.  43.  44.  45.  46.  47.  48.]
 [ 49.  50.  51.  52.  53.  54.  55.  56.  57.  58.]
 [ 59.  60.  61.  62.  63.  64.  65.  66.  67.  68.]
 [ 69.  70.  71.  72.  73.  74.  75.  76.  77.  78.]
 [ 79.  80.  81.  82.  83.  84.  85.  86.  87.  88.]
 [ 89.  90.  91.  92.  93.  94.  95.  96.  97.  98.]
 [ 99. 100. 101. 102. 103. 104. 105. 106. 107. 108.]]


In [3]:
@jit
def use_pandas(a): # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a) # Numba doesn't know about pd.DataFrame
    df += 1                        # Numba doesn't understand what this is
    return df.cov()                # or this!

print(use_pandas(x))

            0           1           2           3           4           5  \
0  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   
1  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   
2  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   
3  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   
4  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   
5  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   
6  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   
7  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   
8  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   
9  916.666667  916.666667  916.666667  916.666667  916.666667  916.666667   

            6           7           8           9  
0  916.666667  916.666667  916.666667  916.666667  
1  916.666667  916.666667  916.666667  916.66666

<ipython-input-3-7e9780beb20f>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "use_pandas" failed type inference due to: Unknown attribute 'DataFrame' of type Module(<module 'pandas' from '/home/mephisto/anaconda3/envs/nu_numba/lib/python3.8/site-packages/pandas/__init__.py'>)

File "<ipython-input-3-7e9780beb20f>", line 3:
def use_pandas(a): # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a) # Numba doesn't know about pd.DataFrame
    ^

During: typing of get attribute at <ipython-input-3-7e9780beb20f> (3)

File "<ipython-input-3-7e9780beb20f>", line 3:
def use_pandas(a): # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a) # Numba doesn't know about pd.DataFrame
    ^

  @jit
/home/mephisto/anaconda3/envs/nu_numba/lib/python3.8/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "use_pandas" was compiled in object mode without forceobj=True.

File "<ipyth

從上面的運行結果可以發現，Numba 無法應用在 Pandas 上，所以會出警告訊息

## 2. What is `nopython` mode?

從上面 NumPy 的例子可以發現說，在裝飾器 `@jit` 後面有個 `nopython` 參數的選擇，簡要來說，當設置為 `true` 時，會將你的程式碼直接轉成機器語言，藉此提升運算速度。

## 3. How to measure the performance of Numba?

下面是用 `time` 去測量 Numba 的運算效率 ： 

In [4]:
x = np.arange(100).reshape(10, 10)

@jit(nopython=True)
def go_fast(a):  # Function is compiled and runs in machine code
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.perf_counter()
go_fast(x)
end = time.perf_counter()
print("Elapsed (with compilation) = {}s".format((end - start)))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.perf_counter()
go_fast(x)
end = time.perf_counter()
print("Elapsed (after compilation) = {}s".format((end - start)))

Elapsed (with compilation) = 0.07662968899990119s
Elapsed (after compilation) = 2.177999999730673e-05s


從上面可以看到，只要使用 Numba 編譯過一次，在藉由緩存再次計算的程式效率。

## 4. NumPy v.s. Numba

這邊來比較一下 NumPy 和 Numba 的效率 ： 

In [5]:
t = np.linspace(1, 1000000, 2000000)

def numpy_do_numerical(x):
    tmp = 0.0
    for i in range(x.shape[0]):
        tmp += np.sin(i)
    return np.sum(x) + tmp

@jit(nopython=True)
def numba_do_numerical(x):
    tmp = 0.0
    for i in range(x.shape[0]):
        tmp += np.sin(i)
    return np.sum(x) + tmp


time_0 = time.time()
numpy_test = numpy_do_numerical(t)
time_1 = time.time()
numba_test = numba_do_numerical(t)
time_2 = time.time()

numpy_time_cost = time_1 - time_0
numba_time_cost = time_2 - time_1

print(f'NumPy time cost : {numpy_time_cost}')
print(f'Numba time cost : {numba_time_cost}')

NumPy time cost : 0.9824647903442383
Numba time cost : 0.11569905281066895
